In [4]:
import os
from PIL import Image
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.metrics.pairwise import cosine_similarity
import csv
import chardet


# Carregar o modelo MobileNetV2 pré-treinado
print("Carregando modelo MobileNetV2...")
model_path = "MobileNetV2_model.h5"
base_model = MobileNetV2(weights=model_path, include_top=False, input_shape=(224, 224, 3))
print("Modelo MobileNetV2 carregado com sucesso.")

# Função para determinar a codificação do arquivo CSV
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
    return encoding

# Carregar características dos pets a partir do arquivo CSV
pet_features = np.loadtxt('pet_features.csv', delimiter=',')
print("pet_features carregadas com sucesso.")

# Carregar informações dos arquivos dos pets a partir do arquivo CSV
pet_files_path = 'pet_files.csv'
csv_encoding = detect_encoding(pet_files_path)
with open(pet_files_path, 'r', encoding=csv_encoding) as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Ignorar o cabeçalho
    pet_files = [row[0] for row in reader]

print("pet_files carregadas com sucesso.")


# Função para encontrar o pet mais semelhante
def find_most_similar_pet(query_features):
    similarities = cosine_similarity([query_features], pet_features)
    most_similar_indices = similarities.argsort()[0][-20:][::-1]
    most_similar_files = [pet_files[i] for i in most_similar_indices]

    # Formatar os nomes dos arquivos
    formatted_most_similar_files = []
    for file in most_similar_files:
        # Extrair o número do ID do arquivo
        #id_number = file.split('_id_')[1].split('_')[0]
        id_number = file.split('.')[0]
        # Formatar a URL com base no número do ID
        url = f"https://petsrs.com.br/pet/{id_number}"
        formatted_most_similar_files.append(url)

    return formatted_most_similar_files

Carregando modelo MobileNetV2...
Modelo MobileNetV2 carregado com sucesso.
pet_features carregadas com sucesso.
pet_files carregadas com sucesso.


In [5]:
# Testar com uma imagem de exemplo
query_image_path = r'C:\Users\Pichau\Downloads\gato.jpg'
img = Image.open(query_image_path).resize((224, 224))
img = np.array(img)
img = preprocess_input(img)
query_features = base_model.predict(np.expand_dims(img, axis=0)).flatten()
most_similar_pets = find_most_similar_pet(query_features)
print("Pet similares:", most_similar_pets)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 929ms/step
Pet similares: ['https://petsrs.com.br/pet/1501', 'https://petsrs.com.br/pet/1510', 'https://petsrs.com.br/pet/3113', 'https://petsrs.com.br/pet/1511', 'https://petsrs.com.br/pet/1395', 'https://petsrs.com.br/pet/1502', 'https://petsrs.com.br/pet/286', 'https://petsrs.com.br/pet/226', 'https://petsrs.com.br/pet/3835', 'https://petsrs.com.br/pet/2715', 'https://petsrs.com.br/pet/1247', 'https://petsrs.com.br/pet/360', 'https://petsrs.com.br/pet/3797', 'https://petsrs.com.br/pet/2215', 'https://petsrs.com.br/pet/1920', 'https://petsrs.com.br/pet/2639', 'https://petsrs.com.br/pet/1336', 'https://petsrs.com.br/pet/1326', 'https://petsrs.com.br/pet/2899', 'https://petsrs.com.br/pet/3519']
